<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import random

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def save_json(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f)

def split_data(data, train_ratio=0.8):
    # Shuffle the data
    random.shuffle(data)

    # Calculate the split index
    split_index = int(len(data) * train_ratio)
    print(split_index)

    # Split the data
    train_data = data[:split_index]
    test_data = data[split_index:]

    return train_data, test_data

# Load the JSON data
json_file_path = '/content/drive/MyDrive/FYP/Implementation/Resume Dataset/10_resumes_annotated.json'
data = load_json(json_file_path)

# Split the data
train_data, test_data = split_data(data)

# Save the splits
save_json(train_data, 'train_data.json')
save_json(test_data, 'test_data.json')

17


In [3]:
dt = load_json('train_data.json')
for at in dt:
  print(at)

{'id': 18502, 'annotations': [{'id': 15, 'completed_by': 1, 'result': [{'value': {'start': 37, 'end': 49, 'text': 'bob markkula', 'labels': ['NAME']}, 'id': 'gUiHTX7iyG', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 69, 'end': 79, 'text': '2149270369', 'labels': ['PHONE']}, 'id': '8YPn2_sMHh', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 80, 'end': 100, 'text': 'bobqmarkkulagmailcom', 'labels': ['EMAIL']}, 'id': 'OSDcT6_bGd', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 133, 'end': 151, 'text': 'database developer', 'labels': ['JOB']}, 'id': 'Bb2U0kExpW', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 179, 'end': 182, 'text': 'sql', 'labels': ['SKILL']}, 'id': 'H6hvpqtNfs', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start':

In [4]:
dt = load_json('test_data.json')
for at in dt:
  print(at)

{'id': 18498, 'annotations': [{'id': 11, 'completed_by': 1, 'result': [{'value': {'start': 0, 'end': 16, 'text': 'harrison sanford', 'labels': ['NAME']}, 'id': 'CyQSEyacvB', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 37, 'end': 49, 'text': 'philadelphia', 'labels': ['LOC']}, 'id': 'tjjQlH2IPF', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 59, 'end': 73, 'text': '1 555 154 3342', 'labels': ['PHONE']}, 'id': 'qu1C87ZskA', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 85, 'end': 97, 'text': 'devops leads', 'labels': ['JOB']}, 'id': 'PA7pWspv5Z', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'start': 97, 'end': 116, 'text': 'tarbucks technology', 'labels': ['COMPANY']}, 'id': 'WnjM5J7oX6', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}, {'value': {'st

In [22]:
import json
import random
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin

# Load JSON data
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/10_resumes_annotated.json', "r") as f:
    data = json.load(f)

# Function to convert JSON data to Spacy's DocBin format
def convert_to_spacy_format(data):
    nlp = spacy.blank("en")  # Load a blank Spacy model
    doc_bin = DocBin()  # Container for our docs

    for item in data:
        text = item['data']['Text']  # Full document text
        entities = []

        for annotation in item['annotations'][0]['result']:
            start = annotation['value']['start']
            end = annotation['value']['end']
            label = annotation['value']['labels'][0]  # Entity label
            entities.append((start, end, label))

        # Create a Spacy doc and add entities to it
        doc = nlp.make_doc(text)
        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]
        # Filter out None spans if Spacy can't align the character indices with tokens
        spans = [span for span in spans if span is not None]
        doc.ents = spans  # Assign entities to the doc
        doc_bin.add(doc)

    return doc_bin

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert train and test sets to Spacy format
train_doc_bin = convert_to_spacy_format(train_data)
test_doc_bin = convert_to_spacy_format(test_data)

# Save the train and test data to .spacy files
train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")
test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")


## Spacy NER

In [18]:
# create dir to place spacy ner data
!mkdir spacy_ner_data

# convert CONLL2003 annotation data into spacy data
!python -m spacy convert 'train_data.json' spacy_ner_data -c ner
!python -m spacy convert "test_data.json" spacy_ner_data -c ner

mkdir: cannot create directory ‘spacy_ner_data’: File exists
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (23 documents): spacy_ner_data/train.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (7 documents): spacy_ner_data/test.spacy


In [6]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [13]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# train model using hyperparameters set in config.cfg
# trained model in output/ dir
!python -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    296.83    1.01    2.56    0.63    0.01
  9     200       1380.49  12424.58   25.91   36.36   20.13    0.26
 18     400        165.97   3289.83   41.80   60.00   32.08    0.42
 27     600        117.71   1442.75   44.70   56.19   37.11    0.45
 36     800        114.14    847.01   41.98   53.40   34.59    0.42


In [20]:
# evaluate trained model performance
# store output and visualization into result/ dir
!python -m spacy evaluate output/model-best spacy_ner_data/test_data.spacy -dp output

ℹ Using CPU

================================== Results ==================================

TOK     -     
NER P   100.00
NER R   100.00
NER F   100.00
SPEED   75    


=============================== NER (per type) ===============================

          P        R        F
}]   100.00   100.00   100.00

<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/usr/local/lib/python3.10/dist-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/typer/core.py

In [ ]:
# make prediction
import spacy
resume_text = 'dot net developer robert smith phone 123 456 78 99 email infoqwikresumecom website wwwqwikresumecom linkedin linkedincomqwikresume address 1737 marshville road alabama objective dot net developer seven years experience design development webbased windows based applications using net technologies hands experience phases software development life cycle sdlc like requirement gathering analysis architectural detail design documentation development testing implementation using agile methodologies like scrum xp test driven environment skills programming languages net technologies c sql plsql web forms win forms web technologies scripting html dhtml css xmlangular jsbootstrap ajax toolkit jquery javascript telerikkendo ui database ms sql server operating systems windows 8 packages ms office amp visio ms frontpage iis version control tools git hub vss tfs methodologies agile oops scrum soa reporting crystal reports net ms sql server reporting services ssrs work experience dot net developer charter communications june 2015 present assisting developing architectural design functional specifications involving analysis designing coding implementation application developing dynamic web page implemented creatively implemented design requirements using client side scripting language technologies assisting agile software development management activities respond unpredictability iterative sprints designing developing web application migrating project mvc architecture using mvc 3 separate internal representations information involved developing telerik kendo ul controls building application enabling focus value generating development tasks involving development presentation logic gui aspnet pages dot net developer abc corp 2011 2015 coded updated maintained computer programs assisted developers prepare high level technical design documents performed code enhancements assist performance analysis provided technical guidance programming standards team trained users team coordinated client amp offshore tearn meet project objectives participated backlog grooming meeting work client remove barriers team worked individual developer also manage offshore team free resume template copyright qwikresumecom usage guidelines'
nlp = spacy.load("output/model-best")
doc = nlp('sql javascript css xml json jfgrdht jkdsfnsfaetewtjnsgr')

print(doc.ents)

for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

(sql, javascript, css, xml, json, jfgrdht, jkdsfnsfaetewtjnsgr)
sql: }]
javascript: }]
css: }]
xml: }]
json: }]
jfgrdht: }]
jkdsfnsfaetewtjnsgr: }]


In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# download trained model

## Flair NER

In [ ]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f0e874a786bf8d476934747387c310affa5f28c7f6f13f9a45e27dcf2e717558
  Stored in directory: /root/.cache

In [ ]:
import spacy
from spacy.tokens import DocBin
import os

def convert_spacy_to_flair(input_file, output_file):
    """
    Convert SpaCy binary format to Flair's CoNLL format.

    Args:
        input_file (str): Path to SpaCy binary file (.spacy)
        output_file (str): Path to output file for Flair format
    """
    # Load spaCy model
    nlp = spacy.blank("en")

    # Load the DocBin
    doc_bin = DocBin().from_disk(input_file)
    docs = list(doc_bin.get_docs(nlp.vocab))

    with open(output_file, 'w', encoding='utf-8') as f:
        for doc in docs:
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            # Write tokens in CoNLL format
            for token in tokens:
                text, iob, ent_type = token

                # Convert spaCy IOB to CoNLL format
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'

                # Write line: token and NER tag
                f.write(f'{text} {tag}\n')

            # Empty line between sentences
            f.write('\n')

def convert_spacy_json_to_flair(input_file, output_file):
    """
    Convert SpaCy JSON format to Flair's CoNLL format.

    Args:
        input_file (str): Path to JSON file with SpaCy annotations
        output_file (str): Path to output file for Flair format
    """
    import json

    nlp = spacy.blank("en")

    with open(input_file, 'r', encoding='utf-8') as f:
        training_data = json.load(f)

    with open(output_file, 'w', encoding='utf-8') as f:
        for example in training_data:
            text = example['text']
            ents = example.get('entities', [])

            # Create a spaCy doc
            doc = nlp(text)

            # Add entities to doc
            spans = []
            for start, end, label in ents:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    spans.append(span)
            doc.ents = spans

            # Convert to CoNLL format
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            for token in tokens:
                text, iob, ent_type = token
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'
                f.write(f'{text} {tag}\n')

            f.write('\n')

# Example usage for JSON format
flair_train_json = "flair_train.txt"
flair_test_json = "flair_test.txt"

convert_spacy_to_flair('/content/spacy_ner_data/train_data.spacy', flair_train_json)
convert_spacy_to_flair('/content/spacy_ner_data/test_data.spacy', flair_test_json)

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# Define columns for CoNLL (0: word, 1: label)
columns = {0: 'text', 1: 'ner'}

# Set data folder and file names
data_folder = './'
train_file = 'flair_train.txt'
test_file = 'flair_test.txt'

# Load the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=train_file,
                              test_file=test_file)

2024-10-28 15:25:01,956 Reading data from .
2024-10-28 15:25:01,959 Train: flair_train.txt
2024-10-28 15:25:01,965 Dev: None
2024-10-28 15:25:01,967 Test: flair_test.txt
2024-10-28 15:25:01,973 No dev split found. Using 10% (i.e. 0 samples) of the train split as dev data


In [ ]:
# create NER tagger
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger

embeddings = StackedEmbeddings([
                WordEmbeddings('glove'),
                WordEmbeddings('en-crawl')
            ])

tagger = SequenceTagger(hidden_size=256,
                         embeddings=embeddings,
                         tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),
                         tag_type='ner',
                         use_crf=True)

2024-10-28 15:35:21,264 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>


<ipython-input-71-687232df04ba>:12: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),


In [ ]:
# train model
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

trainer = ModelTrainer(tagger, corpus)

trainer.train('flair_output/',  # Update this path
               learning_rate=0.1,
               mini_batch_size=32,
               max_epochs=10)

trainer.train(
     base_path='/flair_output/',
     learning_rate=0.1,
     mini_batch_size=32,
     max_epochs=10,
     patience=3,
     embeddings_storage_mode='gpu',
    #  checkpoint=True,
     use_amp=True,  # Use mixed precision training
     train_with_dev=False,
    #  monitor_train=True,
    #  monitor_test=True,
     evaluation_metric=EvaluationMetric.MICRO_F1_SCORE,
)

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:84: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2024-10-28 15:37:39,042 ----------------------------------------------------------------------------------------------------
2024-10-28 15:37:39,048 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): WordEmbeddings(
      'en-crawl'
      (embedding): Embedding(1000001, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=400, out_features=400, bias=True)
  (rnn): LSTM(400, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2024-10-28 15:37:39,050 ----------------------------------------------------------------------------------------------------
2024-10-28 15:37:39,056 Corpus: 1 train + 0 dev + 1 test sentences
2024-10-28 15:37:39,058 --------------------------------------

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-10-28 15:38:42,967 ----------------------------------------------------------------------------------------------------
2024-10-28 15:38:42,984 epoch 2 - iter 1/1 - loss 0.00000000 - time (sec): 0.01 - samples/sec: 73.39 - lr: 0.100000 - momentum: 0.000000
2024-10-28 15:38:42,987 ----------------------------------------------------------------------------------------------------
2024-10-28 15:38:42,989 EPOCH 2 done: loss 0.0000 - lr: 0.100000
2024-10-28 15:38:42,992  - 1 epochs without improvement
2024-10-28 15:38:42,994 ----------------------------------------------------------------------------------------------------
2024-10-28 15:38:43,008 epoch 3 - iter 1/1 - loss 0.00000000 - time (sec): 0.01 - samples/sec: 87.57 - lr: 0.100000 - momentum: 0.000000
2024-10-28 15:38:43,010 ----------------------------------------------------------------------------------------------------
2024-10-28 15:38:43,014 EPOCH 3 done: loss 0.0000 - lr: 0.100000
2024-10-28 15:38:43,016  - 2 epochs with

100%|██████████| 1/1 [00:00<00:00, 133.50it/s]

2024-10-28 15:39:49,090 It was not possible to compute evaluation values because: 
- The evaluation data has no gold labels for label_type='ner'!
- And no predictions were made!
Double check your corpus (if the test split has labels), and how you initialize the ModelTrainer!
2024-10-28 15:39:49,094 ----------------------------------------------------------------------------------------------------


TypeError: SGD.__init__() got an unexpected keyword argument 'monitor_train'

In [ ]:
# evaluate model
result, score = trainer.evaluate(corpus.test)
print(result)
print(score)

In [ ]:
# make prediction
model = SequenceTagger.load('path/to/save/model')
sentence = flair.data.Sentence("Your text here.")

model.predict(sentence)

print(sentence.to_tagged_string())